In [1]:
import os
import pandas as pd
import math

from IPython import get_ipython

%reload_ext sparkmagic.magics

In [9]:
username = os.environ['RENKU_USERNAME']
server = "http://iccluster029.iccluster.epfl.ch:8998"

# set the application name as "<your_gaspar_id>-homework3"
get_ipython().run_cell_magic(
    'spark',
    line='config', 
    cell="""{{ "name": "{0}-finalproject", "executorMemory": "4G", "executorCores": 4, "numExecutors": 10, "driverMemory": "4G"}}""".format(username)
)

In [10]:
get_ipython().run_line_magic(
    "spark", "add -s {0}-finalproject -l python -u {1} -k".format(username, server)
)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
8555,application_1652960972356_4325,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


In [81]:
%%spark
stop_times = spark.read.csv("/data/sbb/csv/stop_times/2019/05/08/stop_times.txt", sep=',', header='true')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
%%spark
import pyspark.sql.functions as F

dep_tbl = stop_times.select([
    F.col('trip_id'),
    F.col('departure_time'),
    F.col('stop_id').alias('dep_stop_id'),
    (F.col('stop_sequence').cast('int') + 1).alias('stop_sequence'),
])

arr_tbl = stop_times.select([
    F.col('trip_id'),
    F.col('arrival_time'),
    F.col('stop_id').alias('arr_stop_id'),
    F.col('stop_sequence').cast('int').alias('stop_sequence'),
])

timetable = dep_tbl.join(arr_tbl, ['trip_id', 'stop_sequence']).select([
    F.col('trip_id'),
    F.col('dep_stop_id').alias('dep_stop_id'),
    F.col('arr_stop_id').alias('arr_stop_id'),
    F.col('departure_time').alias('dep_time'),
    F.col('arrival_time').alias('arr_time'),
])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
%%spark
#-o final_timetable -n 10000

START_TIME = 6
END_TIME = 22
# START_TIME = 6
# END_TIME = 22

timetable = timetable.withColumn("dep_time", F.to_timestamp("dep_time", "HH:mm:ss")) \
.withColumn("arr_time", F.to_timestamp("arr_time", "HH:mm:ss"))

final_timetable = timetable.filter(
      (F.hour(timetable.dep_time) >= START_TIME) & ((F.hour(timetable.arr_time) <= END_TIME))
    ).withColumn("dep_time", F.concat(F.hour(F.col('dep_time')),F.lit(':'), F.minute(F.col('dep_time')))) \
    .withColumn("arr_time", F.concat(F.hour(F.col('arr_time')),F.lit(':'), F.minute(F.col('arr_time')))).withColumn("mean",F.lit(0)).withColumn("std",F.lit(0))

final_timetable.show(10)
#not working
#final_timetable.rdd.saveAsPickleFile('../data/timetable.pkl')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+-----------+--------+--------+----+---+
|             trip_id|dep_stop_id|arr_stop_id|dep_time|arr_time|mean|std|
+--------------------+-----------+-----------+--------+--------+----+---+
|1.TA.1-357-A-j19-...|    8572495|    8503593|   20:34|   20:35|   0|  0|
|1.TA.1-371-j19-1.1.H|    8572475|    8500979|    6:17|    6:18|   0|  0|
|1.TA.11-13-A-j19-...|8574694:0:E|8582197:0:E|   12:51|   12:51|   0|  0|
|1.TA.11-17-j19-1.1.R|    8582909|    8582908|    8:30|    8:31|   0|  0|
|1.TA.11-202-A-j19...|    8573635|    8583411|   15:52|   15:53|   0|  0|
|1.TA.11-22-j19-1.1.R|8575114:0:E|    8509697|   11:25|   11:27|   0|  0|
|1.TA.11-401-j19-1...|    8574606|    8574605|   11:36|   11:38|   0|  0|
|1.TA.11-411-j19-1...|    8581051|    8509728|   15:58|   15:59|   0|  0|
|1.TA.11-42-j19-1.1.R|    8574493|    8575112|    7:37|    7:41|   0|  0|
|1.TA.11-702-j19-1...|    8583094|    8579989|   20:14|   20:16|   0|  0|
+--------------------+-----------+----

In [84]:
%%spark
final_timetable.filter(final_timetable.trip_id=="105.TA.26-16-A-j19-1.56.H").show(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------+---------------+--------+--------+----+---+
|             trip_id|    dep_stop_id|    arr_stop_id|dep_time|arr_time|mean|std|
+--------------------+---------------+---------------+--------+--------+----+---+
|105.TA.26-16-A-j1...|    8503101:0:3|    8503141:0:2|    9:31|    9:32|   0|  0|
|105.TA.26-16-A-j1...|    8503111:0:1|    8503102:0:2|    9:26|    9:28|   0|  0|
|105.TA.26-16-A-j1...|    8503102:0:2|    8503101:0:3|    9:28|    9:31|   0|  0|
|105.TA.26-16-A-j1...|    8503103:0:2|    8503111:0:1|    9:24|    9:26|   0|  0|
|105.TA.26-16-A-j1...|    8503141:0:2|    8503100:0:3|    9:32|    9:34|   0|  0|
|105.TA.26-16-A-j1...|    8503020:0:3|    8503006:0:8|    9:48|    9:53|   0|  0|
|105.TA.26-16-A-j1...|    8503006:0:8|    8503016:0:1|    9:54|    9:59|   0|  0|
|105.TA.26-16-A-j1...|    8503100:0:3|    8503004:0:1|    9:34|    9:37|   0|  0|
|105.TA.26-16-A-j1...|    8503003:0:1|8503000:0:41/42|    9:41|    9:44|   0|  0|
|105.TA.26-16-A-

## Merging/filtering on zurich stops

### Gio chunk

In [85]:
%%spark -o stops_15k_ids -n -1
stops_15k = spark.read.csv("/user/monopoli/stops_15k/stops_15k.csv", sep=',', header='true')
stops_15k_ids = stops_15k.drop('_c0', 'stop_name')
list_of_stops_in_15k = list(stops_15k_ids.select('stop_id').toPandas()['stop_id'])
list_of_stops_in_15k = [stop.encode("utf-8") for stop in list_of_stops_in_15k]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [86]:
%%spark
#delete dep stop_id outside 15km area
final_timetable = final_timetable.filter(final_timetable.dep_stop_id.isin(list_of_stops_in_15k))

#delete arrival stop_id outside 15km area
final_timetable = final_timetable.filter(final_timetable.arr_stop_id.isin(list_of_stops_in_15k))


#delete direct connections where it starts same as where it arrives
final_timetable = final_timetable.filter(final_timetable.dep_stop_id != final_timetable.arr_stop_id)

#delete duplicates
#final_timetable = final_timetable.dropDuplicates(['dep_stop_id','arr_stop_id','dep_time','arr_time'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Calendar information

In [87]:
%%spark
cal = spark.read.csv("/data/sbb/csv/calendar/2019/05/08/calendar.txt", sep=',', header='true')
trips = spark.read.csv("/data/sbb/csv/trips/2019/05/08/trips.txt", sep=',', header='true')
merged_cal = trips.join(cal,on="service_id") \
.drop("trip_short_name","start","service_id","direction_id","start_date","end_date")
merged_all = final_timetable.join(merged_cal,on="trip_id")
final_timetable = merged_all

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [88]:
%%spark
routes_df = spark.read.csv("/data/sbb/csv/routes/2019/05/08/routes.txt", sep=',', header='true')
final_timetable = final_timetable.join(routes_df,on="route_id").drop("route_id","agency_id","route_long_name","route_type")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
%%spark -o len_df

#get lenght of dataframe
len_df =spark.createDataFrame(
    [
        (1,final_timetable.count()),
    ],
    ["id","count"]
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
%%spark
final_timetable.show(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------+-----------+--------+--------+----+---+--------------------+------+-------+---------+--------+------+--------+------+----------------+----------+
|             trip_id|    dep_stop_id|arr_stop_id|dep_time|arr_time|mean|std|       trip_headsign|monday|tuesday|wednesday|thursday|friday|saturday|sunday|route_short_name|route_desc|
+--------------------+---------------+-----------+--------+--------+----+---+--------------------+------+-------+---------+--------+------+--------+------+----------------+----------+
|1.TA.26-18-j19-1.1.H|        8503064|8503065:0:1|   10:41|   10:45|   0|  0|Zürich Stadelhofe...|     1|      1|        1|       1|     1|       0|     0|              18|    S-Bahn|
|1.TA.26-18-j19-1.1.H|        8530812|    8530813|    11:2|    11:4|   0|  0|Zürich Stadelhofe...|     1|      1|        1|       1|     1|       0|     0|              18|    S-Bahn|
|1.TA.26-18-j19-1.1.H|        8530811|    8530812|   10:59|    11:2|   0|  0|Zür

In [96]:
%%spark
final_timetable.coalesce(1).write.option("header","true").format("csv").save('/group/mmcc/final_hdfs_timetable3.csv')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
len_df

,id,count
0,1,1670414


Index whole dataframe

In [47]:
%%spark
from pyspark.sql.window import Window
w = Window().orderBy(F.lit('A'))
final_timetable = final_timetable.withColumn("rowNum", F.row_number().over(w))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
%%spark
final_timetable.select("rowNum").distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1670414

In [73]:
%%spark
test = final_timetable.filter((final_timetable.rowNum >= 0) & (final_timetable.rowNum <=20))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
%%spark -o test -n -1
test

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[trip_id: string, dep_stop_id: string, arr_stop_id: string, dep_time: string, arr_time: string, mean: int, std: int, trip_headsign: string, monday: string, tuesday: string, wednesday: string, thursday: string, friday: string, saturday: string, sunday: string, route_short_name: string, route_desc: string, rowNum: int]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
test

,trip_id,dep_stop_id,arr_stop_id,dep_time,arr_time,mean,std,trip_headsign,monday,tuesday,wednesday,thursday,friday,saturday,sunday,route_short_name,route_desc,rowNum
0,10.TA.26-311-j19-1.1.R,8590619,8590525,2022-05-29 09:27:00,2022-05-29 09:28:00,0,0,"Dietikon, Bahnhof",0,0,0,0,0,0,1,311,Bus,15
1,10.TA.26-311-j19-1.1.R,8590529,8587020:0:D,2022-05-29 09:29:00,2022-05-29 09:30:00,0,0,"Dietikon, Bahnhof",0,0,0,0,0,0,1,311,Bus,16
2,10.TA.26-311-j19-1.1.R,8590845,8590837,2022-05-29 09:20:00,2022-05-29 09:21:00,0,0,"Dietikon, Bahnhof",0,0,0,0,0,0,1,311,Bus,17
3,10.TA.26-311-j19-1.1.R,8590800,8590801,2022-05-29 09:22:00,2022-05-29 09:23:00,0,0,"Dietikon, Bahnhof",0,0,0,0,0,0,1,311,Bus,18
4,10.TA.26-311-j19-1.1.R,8590834,8590849,2022-05-29 09:16:00,2022-05-29 09:17:00,0,0,"Dietikon, Bahnhof",0,0,0,0,0,0,1,311,Bus,19
5,10.TA.26-311-j19-1.1.R,8590842,8590845,2022-05-29 09:19:00,2022-05-29 09:20:00,0,0,"Dietikon, Bahnhof",0,0,0,0,0,0,1,311,Bus,20


In [74]:
#do the chunk

path = os.path.join("","../data/test.csv")
try:
    os.remove(path)
except:
    pass

lenght_df = 20
counter = 0
chunk_nr = 5
for i in range(0,math.ceil(lenght_df/chunk_nr)):
    print(math.ceil(lenght_df/chunk_nr))
    print(counter)
    #get_ipython().run_line_magic('spark', '-o final_timetable --maxrows 50000')
    get_ipython().run_cell_magic(u'spark', u'-o test -n 5', u'test = test.filter(test.rowNum >= {0})'.format(counter))
    counter+=chunk_nr
    test.to_csv('../data/test.csv', mode='a', header=False)

4
0


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4
5


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

...

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4
15


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
#do the chunk

path = os.path.join("","../data/final_timetable.csv")
try:
    os.remove(path)
except:
    pass

lenght_df = len_df.iloc[0][1]
counter = 0
chunk_nr = 100000
for i in range(0,math.ceil(lenght_df/chunk_nr)):
    print(math.ceil(lenght_df/chunk_nr))
    print(counter)
    #get_ipython().run_line_magic('spark', '-o final_timetable --maxrows 50000')
    get_ipython().run_cell_magic(u'spark', u'-o final_timetable --maxrows 100000', u'final_timetable = final_timetable.filter(final_timetable.rowNum >= {0})'.format(counter))
    counter+=chunk_nr
    final_timetable.to_csv('../data/final_timetable.csv', mode='a', header=False)

17
0


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

17
100000


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

...

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

17
1600000


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#read csv
final_df = pd.read_csv('../data/final_timetable.csv')

In [3]:
#melt it by adding day of week
inter = final_df.melt(id_vars=['trip_id','dep_stop_id','arr_stop_id','dep_time','arr_time','mean','std','trip_headsign','route_desc',"route_short_name"], value_vars=['monday','tuesday','wednesday','thursday','friday','saturday','sunday'])
final_inter = inter[inter.value != 0].copy()


map_days = {"monday" : 1, "tuesday": 2, "wednesday" : 3, 'thursday': 4, "friday":5,"saturday":6,"sunday":7}
final_inter['dayofweek'] = final_inter['variable'].map(map_days)

In [4]:
#parse time cause of bug in spark

def parseBetterTime(time_str):
    split = time_str.split(":")
    if len(split[1]) < 2:
        return split[0] + ":0" + split[1]
    else:
        return time_str
final_inter.dep_time = final_inter.dep_time.apply(parseBetterTime)
final_inter.arr_time = final_inter.arr_time.apply(parseBetterTime)

In [5]:
#clean and send to csv
final_inter = final_inter.reset_index(drop=True)
final_inter.rename(columns={"route_desc": "transport_name"},inplace=True)
final_inter = final_inter[['trip_id','dep_stop_id','arr_stop_id','dep_time','arr_time','mean','std',"dayofweek","transport_name",'trip_headsign','route_short_name']]
final_inter.to_csv("../data/final_micheal_timetable.csv")